In [ ]:
df = pd.read_csv("diabetes.csv")

scaler = StandardScaler() #PCA를 위한 scaling
X = df.drop(["Outcome"], axis = 1) 
y = df.Outcome

X_scaled = scaler.fit_transform(X)

pca = PCA(n_components = 2) #거리 공식 사용 및 시각화에 용의하게 하기 위해 pca 적용
X_pca = pca.fit_transform(X_scaled)

ndf= pd.DataFrame(X_pca)
ndf["Outcome"] = y

ndf.columns=['x','y', 'Outcome']

model = RandomForestClassifier()

X = ndf.drop(["Outcome"], axis = 1)
y = ndf.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

model.fit(X_train, y_train)
pred = model.predict(X_test)


print(accuracy_score(pred, y_test))

In [ ]:
kmeans = KMeans(n_clusters=2,init='k-means++',n_init=10,max_iter=300,random_state=0)
temp = kmeans.fit(X)
ndf["cluster"] = -1
ndf["cluster"] = temp.labels_
cluster0_center, cluster1_center = temp.cluster_centers_[0], temp.cluster_centers_[1]

df_zero = ndf[ndf.cluster == 0]
df_one = ndf[ndf.cluster == 1]
df_zero["dist"] = 10000000
df_one["dist"] = 10000000

for i in range(len(df_zero)):
    df_zero["dist"].iloc[i] = ((cluster0_center[0] - df_zero.iloc[i]['x'])**2 + 
                               (cluster0_center[1] - df_zero.iloc[i]['y'])**2)**(1/2)
for i in range(len(df_one)):
    df_one["dist"].iloc[i] = ((cluster1_center[0] - df_one.iloc[i]['x'])**2 + 
                              (cluster1_center[1] - df_one.iloc[i]['y'])**2)**(1/2)

In [ ]:
sort_dfzero=df_zero.sort_values("dist", ascending = False)
zero_a, zero_b = sort_dfzero['x'].iloc[0] , sort_dfzero['y'].iloc[0]
zero_maxdist=sort_dfzero['dist'].iloc[0]

sort_dfone=df_one.sort_values("dist", ascending = False)
one_a, one_b = sort_dfone['x'].iloc[0] , sort_dfone['y'].iloc[0]
one_maxdist=sort_dfone['dist'].iloc[0]

In [ ]:
zero_top = cluster0_center[1] + abs(zero_b)
zero_bot = cluster0_center[1] - abs(zero_b)
zero_right = cluster0_center[0] + abs(zero_a)
zero_left = cluster0_center[0] - abs(zero_a)

import random
lst_zero_x, lst_zero_y = [], []
while True:
    x = random.uniform(zero_left, zero_right)
    y = random.uniform(zero_bot, zero_top)
    if ((x - cluster0_center[0])**2 + (y - cluster0_center[1])**2)**(1/2) > zero_maxdist:
        continue
    else:
        lst_zero_x.append(x)
        lst_zero_y.append(y)
    if len(lst_zero_x) == 800:
        break

temp1 = pd.DataFrame(lst_zero_x)
temp2 = pd.DataFrame(lst_zero_y)
temp = pd.concat([temp1, temp2], axis = 1)
temp.columns = ['x', 'y']
temp["Outcome"] = 0
temp["cluster"] = 0

In [ ]:
one_top = cluster1_center[1] + abs(one_b)
one_bot = cluster1_center[1] - abs(one_b)
one_right = cluster1_center[0] + abs(one_a)
one_left = cluster1_center[0] - abs(one_a)

import random
lst_one_x, lst_one_y = [], []
while True:
    x = random.uniform(one_left, one_right)
    y = random.uniform(one_bot, one_top)
    if ((x - cluster1_center[0])**2 + (y - cluster1_center[1])**2)**(1/2) > one_maxdist:
        continue
    else:
        lst_one_x.append(x)
        lst_one_y.append(y)
    if len(lst_one_x) == 800:
        break
        
temp1 = pd.DataFrame(lst_one_x)
temp2 = pd.DataFrame(lst_one_y)
ntemp = pd.concat([temp1, temp2], axis = 1)
ntemp.columns = ['x', 'y']
ntemp["Outcome"] = 1
ntemp["cluster"] = 1

nntemp = pd.concat([temp, ntemp], axis = 0)
nntemp.reset_index(inplace = True)
nntemp.drop(['index'], axis = 1, inplace = True)

In [ ]:
df = pd.concat([ndf, nntemp], axis = 0)
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

X = df.drop(["Outcome", "cluster"], axis = 1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)
model = RandomForestClassifier()
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [ ]:
acc = accuracy_score(y_test, pred)
print(acc)